In [1]:
import requests
import pandas as pd
import json
import re
import openai
import numpy as np
from datetime import date

In [2]:
password = pd.read_csv("passwords.csv")
password_nps = password["password"][0]

In [3]:
#pull API using API key
alerts_url = "https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=" + password_nps

response_API = requests.get(alerts_url)
#print(response_API.status_code)
data = response_API.text
alerts_json = json.loads(data)

In [4]:
data_list_of_dicts = alerts_json['data']
alerts_df = pd.DataFrame(data_list_of_dicts)

In [5]:
alerts_df.to_csv('alerts.csv')

In [5]:
# create new columns for results of each method
 
alerts_with_closure_results = alerts_df
alerts_with_closure_results['Road_Closure_LLM']= None
# alerts_with_closure_results['Road_Closure_RegEx'] = None
# alerts_with_closure_results['Road_Closure_Human'] = None

In [ ]:
with open("C:/Users/Drew.Quinton/Documents/GitHub/NPS-Emerging-Mobility/ARTNAPI.txt", "r", encoding="utf-8") as f:
    openai_api_key = f.read()

client = openai.OpenAI(
    api_key = openai_api_key,
    base_url="http://10.75.42.137:4000/" 
    )

In [ ]:
closure_status = [] # we'll iterate on this list and append the whole thing to the df later

user_prompt = 'tell me if the following is a description of a road closure. If so, respond with "yes", if not, respond with "no" : '

for description in alerts_with_closure_results['description']:
    
    response = client.chat.completions.create(
        model = "GPT-4.1-mini", # model to send to the proxy. One of: GPT-4o-Mini, GPT-4o, o1-Mini, o4-Mini, GPT-4.1, GPT-4.1-mini, GPT-4.1-nano,
        # text-embedding-3-small, text-embedding-3-large, MAI-DS-R1, Llama-3.2-Vision, GPT-5, GPT-5-Mini, GPT-5-Nano , gpt-4o-transcribe
        messages = [
            {
                "role": "system",
                "content": 'You will be given a series of phrases. These phrases are general alerts to users visiting national parks. Your job is to identify, alerts that indicate that a road, lane, or parking lot is closed or reduced in capacity. Do not identify trails that are closed or reduced in capacity.'
            },
            {
                "role": "user",
                "content": user_prompt + description
            }
        ]
    )
    closure_response = response.choices[0].message.content
   
    formatted_closure_response = re.sub(r'[^A-Za-z0-9\s]', '', closure_response.lower())
    closure_status.append(formatted_closure_response)
    print(formatted_closure_response)

n = len(closure_status)

alerts_with_closure_results.iloc[:n, alerts_with_closure_results.columns.get_loc("Road_Closure_LLM")] = closure_status
    


NameError: name 'date' is not defined

In [9]:
road_labels = ['road', 'drive', 'bypass', 'highway', 'parking lot', 'parkway', 'route', 'street', 'bridge',
               'roads', 'drives', 'bypasses', 'highways', 'parking lots', 'parkways', 'routes', 'streets', 'bridges']
closed_labels = ['closed', 'closure', 'closures', 'delay', 'delays']

road_pattern = "|".join(map(re.escape, road_labels))  # Regex pattern to find all cells with a road label
closed_pattern = "|".join(map(re.escape, closed_labels))  # Regex pattern to find all cells with a closed label

road_label_matches = alerts_with_closure_results['description'].str.contains(road_pattern, case=False, na=False)
closure_label_matches = alerts_with_closure_results['description'].str.contains(closed_pattern, case=False, na=False)

alerts_with_closure_results["Road_Closure_RegEx"] = (road_label_matches & closure_label_matches).map({True: 'yes', False: 'no'}) # add a column in the alerts dataframe and fill it with 'yes'
                                                                                                                                   # if it matches _both_ road and closure labels, and 'no' if not

In [ ]:
alerts_with_closure_results.to_csv('Alerts_with_closures_' + str(todays_date) + '.csv', index=False)

# stopping here for now as to not re-run the API call and the LLM stuff
# figure out how we want to process this and save it
# iterate below here until you nail it down
# then comment out this whole cell and the following

In [ ]:
# start here when iterating on how to process this
alerts_with_closure_results = pd.read_csv('Alerts_with_closures_' + str(todays_date) + '.csv')

In [ ]:
alerts_with_closure_results['Road_Closure_LLM'].value_counts()

Road_Closure_LLM
no     737
yes    111
Name: count, dtype: int64

In [51]:
sum(alerts_with_closure_results['relatedRoadEvents']=="[]")

826

In [50]:
alerts_with_closure_results[alerts_with_closure_results['Road_Closure_LLM']=="yes"]

,id,url,title,parkCode,description,category,relatedRoadEvents,lastIndexedDate,Road_Closure_LLM,Road_Closure_RegEx
1,55B4D4D1-47F0-454E-ACBB-5EC8D70A084F,NaN,Visitor Centers Open Through November 2,shen,The Bryd Visitor Center and Dickey Ridge Visit...,Information,[],2025-10-20 00:00:00.0,yes,yes
6,D092610A-AD5D-45F8-B8D9-A283FA55D7F2,NaN,What to Expect During Government Funding Lapse,sagu,The following are not currently open or access...,Park Closure,[],2025-10-17 00:00:00.0,yes,no
9,4D18B12B-E625-4D7C-825A-C779A6EACCA5,https://www.co.jefferson.wa.us/CivicAlerts.asp...,South Shore Quinault Road Closed Outside of Pa...,olym,South Shore Road – which is maintained by Jeff...,Information,[],2025-10-17 00:00:00.0,yes,yes
11,2D6862D7-9E2F-42EB-87CB-922FEB8A77E7,NaN,Hurricane Ridge Rd will close at 5:00pm on 10/...,olym,The Hurricane Ridge Road will close at 5:00pm ...,Caution,[],2025-10-17 00:00:00.0,yes,no
12,BF786513-04CA-4234-8A9F-A3581363D747,https://www.nps.gov/olym/planyourvisit/current...,Temporary bridge on North Shore Quinault Rd to...,olym,Olympic National Park crews will remove a temp...,Information,[],2025-10-17 00:00:00.0,yes,yes
...,...,...,...,...,...,...,...,...,...,...
779,1ED85C21-EB71-4CBE-80A9-4A889494FA4E,NaN,Temporary Blackland Prairie Trail Closure to E...,natt,Heavy rain has undermined a culvert making it ...,Park Closure,[],2023-03-08 14:47:37.0,yes,yes
793,89E8F214-29EB-4BF5-B85E-01A4212E4F69,https://www.nps.gov/dena/planyourvisit/conditi...,Park Operations Altered Due to Pretty Rocks La...,dena,Summer access to facilities and services in De...,Park Closure,[],2022-09-19 14:11:22.0,yes,yes
795,7EF3C4AE-4A5C-432A-83D3-D62231362FD1,https://www.nps.gov/articles/monsoon-season.htm,Monsoon Season & Flash Floods,care,"Heavy rains can cause sudden flooding, especia...",Caution,[],2022-07-12 15:11:21.0,yes,yes
807,76963DF5-FF4B-4C00-8F86-2ECA0AED7446,https://www.nps.gov/pinn/planyourvisit/hours.htm,High Crowds & Park Capacity,pinn,High visitation may require limiting access to...,Information,[],2022-01-19 14:37:31.0,yes,no


In [54]:
len(alerts_with_closure_results[(alerts_with_closure_results['relatedRoadEvents']=="[]") & (alerts_with_closure_results['Road_Closure_LLM']=="yes")])

90

In [45]:
len(alerts_with_closure_results[(alerts_with_closure_results['relatedRoadEvents']!="[]") & (alerts_with_closure_results['Road_Closure_LLM']=="yes")])

21

In [28]:
mask = alerts_with_closure_results['Road_Closure_LLM'].eq('yes')
unique_parks = alerts_with_closure_results.loc[mask, 'parkCode'].unique()
print(unique_parks)

['shen' 'sagu' 'olym' 'lewi' 'cavo' 'redw' 'lavo' 'mora' 'fodo' 'glac'
 'cowp' 'kimo' 'gumo' 'glca' 'abli' 'havo' 'stri' 'crla' 'yose' 'vick'
 'grfa' 'grsa' 'grsm' 'noca' 'keaq' 'rocr' 'biso' 'bibe' 'moja' 'lake'
 'chch' 'grte' 'foun' 'blca' 'deva' 'wica' 'gwmp' 'elma' 'nace' 'anac'
 'gree' 'cana' 'orpi' 'natr' 'katm' 'dewa' 'seki' 'cato' 'blrv' 'dino'
 'neri' 'kemo' 'fone' 'pete' 'hale' 'ozar' 'bith' 'cuis' 'long' 'maca'
 'hafe' 'kala' 'isro' 'yell' 'gois' 'hobe' 'shil' 'frla' 'fomr' 'gett'
 'ever' 'guis' 'cave' 'colo' 'anjo' 'vafo' 'york' 'jame' 'buff' 'whsa'
 'laro' 'thro' 'natt' 'dena' 'care' 'pinn' 'chis']


save a new csv file each time we run it
new entry with: date | total number of alerts | total number of closure | total parks with closures | total number of 'related road events' | total parks with 'related road events' | % closures with 'related road event' | % of parks with 'related road events'